# 머신러닝 미리보기

In [24]:
import pandas as pd
from konlpy.tag import Okt
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.lda_model
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 

## 1. LDA(Latent Dirichlet Analysis)

### 1) 데이터 불러오기

In [2]:
reply_data_set = pd.read_csv("data/appreply2.csv", index_col = 0)
reply_data_set

,text,score
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3
...,...,...
995,갑자기 로그아웃 되더니 비밀번호 변경 실패 메세지가 계속 뜨네요. 휴대폰 번호로 인...,1
996,기사님이 상품 픽업을 하셨는지 표시되면 더 좋을 것 같습니다. 가게에서 조리가 늦게...,3
997,요즘 요기요 보다 배민을 많이 쓰는 사람입니다 전화보다 앱을 써서 좀더 간편하고 다...,3
998,취소 됐으면 적어도 전화 주는 제도는 있어야하는거 아닌가요? 주문해놓고 다른거 하는...,1


### 2) 형태소 분석기

In [3]:
okt = Okt()

# 각 문장을 정규표현식으로 전처리한 후, 형태소 분석기를 통해 문장별로 담아라

stopwords = []
sum_text_list = []

for text in reply_data_set["text"] :
    word_list = []
    text = re.sub("[ㄱ-ㅎㅏ-ㅣ\s!@#$%^&*()]", "", text)
    text_list = okt.pos(text, stem = True)
    
    for tex, pos in text_list :
        if (pos in ['Noun', 'Verb']) & (tex not in stopwords) :
            word_list.append(tex)
    
    sum_text_list.append(" ".join(word_list))

sum_text_list
    

<>:10: DeprecationWarning: invalid escape sequence '\s'
<>:10: DeprecationWarning: invalid escape sequence '\s'
C:\Users\user\AppData\Local\Temp\ipykernel_10732\3534246891.py:10: DeprecationWarning: invalid escape sequence '\s'
  text = re.sub("[ㄱ-ㅎㅏ-ㅣ\s!@#$%^&*()]", "", text)


['배달 민족 리뷰 자주 참고 편입 한가지 건의 사항 최신 순 점 뿐 제 주문 자하 메뉴 메뉴 리뷰 확인 하다 기능 메뉴 검색 기능 리뷰 특정 메뉴 검색 기능 문수 않다 메뉴 리뷰 보기 위해 드 래그 하다 시간 소요 되다 비 효율 발생 하다 긍정 검토 해 주심',
 '내 주문 하다 과거 목록 검색 기능 분명 가게 시키다 기억 뭘 시키다 먹다 맞다 알 싶다 찾기 먹다 메뉴 검색 를 곱창 치면 과거 시키다 곱창 목록 뜨다 가게 리뷰 창 리뷰 보기',
 '검색 화면 전체 배달 포장 탭 배달 탭을스 크롤 내리다 볼때 아래 로스 크롤 하다 자꾸만 왼쪽 오른쪽 넘어가다 전체 탭 포장 탭 보다 되다 정말 검색 후 배달 탭 포장 탭 마트 탭뭐 하나 선택 하다 좌우 넘기다 하다 가끔 도아 고스 크롤 한번 내리다 때 왼쪽 넘어가다 전체 탭 가게 되다 쓰다',
 '배달 팁 순 정렬 지역 따르다 추가 배달 료 별도 체크 되어다 배달 팁 하나 누르다 가격 대별 금액 체크 배달 팁 도로 받다 확인 해 야합',
 '최근 업데이트 안드로이드 정도 배민 어플 실행 업데이트 나오다 업데이트 누르다 업데이트 진행 열기 나오다 열기 누르다 업데이트 나오다 무한 반복 삭제 재 설치 환경설정 등 해보다 증상 요 다른 도 증상 보이다 글 보다 이번 업데이트 관련 파일 확인 보이다 참고 사양 폰 자다 되다 보다 핸드폰 사양 문제 이번 업데이트 보이다',
 '찌다 장구 좀 해주다 대체 언제 이면 개선 되다 요 독점 해이 건가 앱 결제 관련 부분 정말 로고 앱 구성 정말 배민 페이 사용 절차 쓰다 것 찌다 놓다 매장 한식 양식 중식 카페 업종 별로 구분 좀 시키다 순 찜 매장 개도 이름 전부 외우다 매번 찾기 서재 검색 거나 주문 하다 목록 찾다 들어가다',
 '앱 실행 시 로그인 여부 먼저 물어보다 비회 이용 선택 하다 경우 나중 입력 선택 후주 결제 시 받다 주소 설정 하다 순서 변경 요 저 때 어플 설치 하다 삭제 설치 자주 펴다 달다 고재 설치 실행 회원정보 주소 저장 을해 놨음 어플 실행 로그인 

### 3) CountVectorizer

In [4]:
vectorizer = CountVectorizer(
    max_df = 0.1,               # 전체 단어 등장 비율이 p 이상인 것만 사용할 것
    min_df = 2,                 # 단어가 적어도 n개 이상인 것만 사용할 것
    max_features = 1000,        # 상위 항목 n개만 사용할 것
    ngram_range = (1,2)         # 단어의 조합
)

In [5]:
feature_vec = vectorizer.fit_transform(sum_text_list)
feature_vec.shape

(1000, 1000)

In [6]:
feature_names = vectorizer.get_feature_names_out()
print(feature_names)

['가가' '가게 검색' '가게 들어가다' '가게 리뷰' '가게 메뉴' '가게 목록' '가게 배달' '가게 별로' '가게 전화'
 '가게 주문' '가격' '가기' '가끔' '가나' '가능' '가다' '가맹' '가면' '가안' '가요' '가원' '가입'
 '가입 하다' '가장' '가족' '가지' '가하다' '갈다' '갈수록' '갈아타다' '감사' '감사하다' '갑자기' '강제'
 '갖다' '개다' '개발' '개발자' '개선' '개선 되다' '개선 되어다' '개선 하다' '개선 해주다' '개인' '개편'
 '갤럭시' '거기' '거나' '거리' '거의' '거지' '건가' '건물' '건의' '건의 사항' '건지다' '건지다 모르다'
 '걸다' '걸리다' '검색 결과' '검색 기능' '검색 되다' '검색 하다' '검토' '겁니다' '겠다' '겪다' '결과' '결재'
 '결제 되다' '결제 수단' '결제 하다' '경우' '경험' '계속' '계정' '고객' '고객 센터' '고려' '고르다' '고민'
 '고배' '고스' '고스 크롤' '고요' '고재' '고치다' '고하' '고해' '공지' '과거' '과정' '관련' '관리' '광고'
 '구분' '구성' '굳이' '그거' '그것' '그게' '그냥' '그대로' '그리다' '근처' '글자' '금액' '금은' '기간'
 '기기' '기능 리뷰' '기능 만들다' '기능 생기다' '기능 추가' '기다' '기다리다' '기도' '기본' '기분' '기사'
 '기사 배달' '기억' '기업' '기요' '기입' '기전' '기존' '기준' '까먹다' '깔다' '깜빡' '꾸다' '끄다' '끊다'
 '끝나다' '끼다' '나가다' '나다' '나서다' '나서다 결제' '나오다' '나중' '남기다' '남다' '내다' '내려가다'
 '내리다' '내용' '넘기다' '넘다' '넘어가다' '넘어가다 않다' '넣다' '네이버' '네이버 로그인' '네이버 아이디'
 '네트워크' '노력' '노출' '노출 되다' '노트' '놓다' '누가' '누구' '누락' '누르다' '누

### 4) LDA

In [7]:
lda = LatentDirichletAllocation(n_components = 5)
lda.fit(feature_vec)

,n_components,5
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [8]:
print(lda.components_.shape)
print(lda.components_)

(5, 1000)
[[ 0.20271936  1.32666969  0.20188746 ...  3.19591823  0.20195719
   0.20000115]
 [ 2.46875663  0.20168929  6.19051588 ...  0.20000438 19.51478113
   8.19858633]
 [ 1.35717706  0.20326046  0.20255606 ...  0.20118596  0.20049463
   0.20000136]
 [ 2.40239923  8.06837742  0.20503547 ...  4.20226151  1.85573055
   0.20065547]
 [ 1.56894772  0.20000313  0.20000513 ...  0.20062991  0.2270365
   0.20075569]]


In [9]:
for idx, topic in enumerate(lda.components_) :
    print(f"# Topic {idx+1}")
    topic_word_idx = topic.argsort()[::-1]
    top_idxs = topic_word_idx[:20]
    topics = [feature_names[idx] for idx in top_idxs]
    print(" ".join(topics))

# Topic 1
쿠폰 이벤트 사진 할인 누르다 가다 들어가다 화면 모르다 생각 바뀌다 처음 정말 쓸다 알다 써다 혜택 정도 진짜 다시
# Topic 2
로그인 인증 다시 깔다 계속 삭제 가입 카드 오류 누르다 문제 번호 아이디 회원 들어가다 설치 핸드폰 네이버 화면 해도
# Topic 3
설정 입력 선택 체크 추가 주소 돼다 만들다 부분 잘쓰다 정렬 수저 바꾸다 계속 이용 사람 사항 만나다 답변 설정 하다
# Topic 4
목록 선택 찾다 개선 자다 추가 사항 확인 크롤 보이다 카테고리 오다 별로 광고 포장 경우 생기다 요청 보기 매장
# Topic 5
취소 고객 시간 전화 기사 시스템 센터 고객 센터 상담 알다 확인 완료 문제 늦다 생각 매장 개선 민족 업체 배달 기사


In [10]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feature_vec, vectorizer)
pyLDAvis.display(vis)

### 5) 문장 살펴보기

In [11]:
sentence_topic = lda.transform(feature_vec)
print(sentence_topic.shape)
print(sentence_topic)

(1000, 5)
[[0.00649372 0.00650559 0.00651995 0.97388781 0.00659293]
 [0.01194749 0.0118358  0.30664839 0.65773947 0.01182885]
 [0.00621137 0.00609739 0.12694956 0.8546339  0.00610779]
 ...
 [0.96730368 0.00807096 0.00829873 0.0081252  0.00820144]
 [0.00959966 0.00962943 0.00963891 0.00957382 0.96155818]
 [0.29852362 0.00549154 0.38983691 0.00548843 0.3006595 ]]


In [12]:
sent_list = []

for n in range(sentence_topic.shape[0]) :
    topic_idx = sentence_topic[n].argmax()
    topic_prob = sentence_topic[n].max()

    sent_list.append([n, topic_idx, topic_prob])

topic_per_data = pd.DataFrame(sent_list, columns = ["no", "topic_no", "prob"])
topic_per_data["review"] = reply_data_set["text"]
topic_per_data

,no,topic_no,prob,review
0,0,3,0.973888,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,..."
1,1,3,0.657739,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...
2,2,3,0.854634,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스..."
3,3,2,0.954835,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...
4,4,1,0.532358,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...
...,...,...,...,...
995,995,1,0.740964,갑자기 로그아웃 되더니 비밀번호 변경 실패 메세지가 계속 뜨네요. 휴대폰 번호로 인...
996,996,4,0.978643,기사님이 상품 픽업을 하셨는지 표시되면 더 좋을 것 같습니다. 가게에서 조리가 늦게...
997,997,0,0.967304,요즘 요기요 보다 배민을 많이 쓰는 사람입니다 전화보다 앱을 써서 좀더 간편하고 다...
998,998,4,0.961558,취소 됐으면 적어도 전화 주는 제도는 있어야하는거 아닌가요? 주문해놓고 다른거 하는...


In [13]:
topic_per_data["topic_no"].value_counts()

topic_no
3    253
4    219
0    184
1    175
2    169
Name: count, dtype: int64

In [14]:
for topic in range(topic_per_data["topic_no"].nunique()) :
    temp_data = topic_per_data.loc[topic_per_data["topic_no"] == topic, : ]
    print(temp_data.sort_values(by = ["prob"], ascending = False).iloc[:5, -1])
    print("=" * 100)

92     바뀐 인터페이스 너무 구려요. 할인이벤트도 잘 안보이고 찜 목록 들어가는 것도 몇 ...
291    가입하고 첫주문해보고나서 이벤트란 둘러보다가 만원할인 받을수있는거 뒤늦게 알았네요....
96     처음 사용 관련한 어떤 쿠폰도 다운로드가 되지 않네요. 10,000원 할인도 우리 ...
273    배달어플의 강점 중 하나는 가게의 별점이 메뉴의 질을 보장한다는 점인데 가게에서 개...
121    << BH6H3MKS >> 어플 정말 잘 쓰고 있고, 쿠폰도 있고 요청사항도 가게랑...
Name: review, dtype: object
492    폰을 바꾸고나서 배민 로그인이 안됩니다. 원래 페북으로 로그인을 하는데 전에 사용하...
370    아니 뭐 어쩌자는 건가요? 핸드폰을 바꿔서 애용하던 배민 다시깔고 로그인하려니까 무...
465    로그인이 안됩니다 이번에 휴대폰 기기를 변경해서 다 초기화 되는 바람에 다시 로그인...
18     최근에 배민 혹은 갤럭시쪽 업데이트에서 문제가 있는것 같습니다. 밑에 다른 리뷰들 ...
823    그런거 없고 설정상에도 자동업데이트 안되어있어요 그냥 휴대인증을 했던걸 앱이 인식을...
Name: review, dtype: object
321    주소를 적었는데 왜 그런지 만약 집이 ♠@2동이라고 치면 계속 ♠@3동으로 표시되서...
737    B마트이용시 라이더요청사항에 ' 요 ' 란 글자안써짐 ㅇ쓰고 아래아ㆍ쓰면 바로 특수...
754    주문시에 수저,포크 체크하는걸 기본적으로 받지않음으로 해놓는게 요즘 트렌드라 그렇게...
128    필터 중복으로 체크할 수 있던 기능은 왜 사라진 건가요? 정렬기준을 하나밖에 선택이...
171    음식 메뉴에 있는 작은 회색 설명 부분이 잘리는게 불편해서 음식의 상세페이지에서라도...
Name: review, dtype: object
59     이 어플의 가장 큰 문제점은 매장 지도확인 유무가 확률적이라는 점입니다. 지도를 시...
808 

## 2. 분류 학습

### 1) 데이터 불러오기

In [15]:
reply_data_set["reaction"] = np.where(reply_data_set["score"] >= 4, 1, 0)
reply_data_set

,text,score,reaction
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4,1
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5,1
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1,0
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2,0
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3,0
...,...,...,...
995,갑자기 로그아웃 되더니 비밀번호 변경 실패 메세지가 계속 뜨네요. 휴대폰 번호로 인...,1,0
996,기사님이 상품 픽업을 하셨는지 표시되면 더 좋을 것 같습니다. 가게에서 조리가 늦게...,3,0
997,요즘 요기요 보다 배민을 많이 쓰는 사람입니다 전화보다 앱을 써서 좀더 간편하고 다...,3,0
998,취소 됐으면 적어도 전화 주는 제도는 있어야하는거 아닌가요? 주문해놓고 다른거 하는...,1,0


### 2) 데이터 전처리

In [16]:
okt = Okt()

# 각 문장을 정규표현식으로 전처리한 후, 형태소 분석기를 통해 문장별로 담아라

stopwords = ["배민", "민족"]
sum_text_list = []

for text in reply_data_set["text"] :
    word_list = []
    text = re.sub("[ㄱ-ㅎㅏ-ㅣ\s!@#$%^&*()]", "", text)
    text_list = okt.pos(text, stem = True)
    
    for tex, pos in text_list :
        if (pos in ['Noun', 'Verb']) & (tex not in stopwords) :
            word_list.append(tex)
    
    sum_text_list.append(" ".join(word_list))

sum_text_list

<>:10: DeprecationWarning: invalid escape sequence '\s'
<>:10: DeprecationWarning: invalid escape sequence '\s'
C:\Users\user\AppData\Local\Temp\ipykernel_10732\1828654728.py:10: DeprecationWarning: invalid escape sequence '\s'
  text = re.sub("[ㄱ-ㅎㅏ-ㅣ\s!@#$%^&*()]", "", text)


['배달 리뷰 자주 참고 편입 한가지 건의 사항 최신 순 점 뿐 제 주문 자하 메뉴 메뉴 리뷰 확인 하다 기능 메뉴 검색 기능 리뷰 특정 메뉴 검색 기능 문수 않다 메뉴 리뷰 보기 위해 드 래그 하다 시간 소요 되다 비 효율 발생 하다 긍정 검토 해 주심',
 '내 주문 하다 과거 목록 검색 기능 분명 가게 시키다 기억 뭘 시키다 먹다 맞다 알 싶다 찾기 먹다 메뉴 검색 를 곱창 치면 과거 시키다 곱창 목록 뜨다 가게 리뷰 창 리뷰 보기',
 '검색 화면 전체 배달 포장 탭 배달 탭을스 크롤 내리다 볼때 아래 로스 크롤 하다 자꾸만 왼쪽 오른쪽 넘어가다 전체 탭 포장 탭 보다 되다 정말 검색 후 배달 탭 포장 탭 마트 탭뭐 하나 선택 하다 좌우 넘기다 하다 가끔 도아 고스 크롤 한번 내리다 때 왼쪽 넘어가다 전체 탭 가게 되다 쓰다',
 '배달 팁 순 정렬 지역 따르다 추가 배달 료 별도 체크 되어다 배달 팁 하나 누르다 가격 대별 금액 체크 배달 팁 도로 받다 확인 해 야합',
 '최근 업데이트 안드로이드 정도 어플 실행 업데이트 나오다 업데이트 누르다 업데이트 진행 열기 나오다 열기 누르다 업데이트 나오다 무한 반복 삭제 재 설치 환경설정 등 해보다 증상 요 다른 도 증상 보이다 글 보다 이번 업데이트 관련 파일 확인 보이다 참고 사양 폰 자다 되다 보다 핸드폰 사양 문제 이번 업데이트 보이다',
 '찌다 장구 좀 해주다 대체 언제 이면 개선 되다 요 독점 해이 건가 앱 결제 관련 부분 정말 로고 앱 구성 정말 페이 사용 절차 쓰다 것 찌다 놓다 매장 한식 양식 중식 카페 업종 별로 구분 좀 시키다 순 찜 매장 개도 이름 전부 외우다 매번 찾기 서재 검색 거나 주문 하다 목록 찾다 들어가다',
 '앱 실행 시 로그인 여부 먼저 물어보다 비회 이용 선택 하다 경우 나중 입력 선택 후주 결제 시 받다 주소 설정 하다 순서 변경 요 저 때 어플 설치 하다 삭제 설치 자주 펴다 달다 고재 설치 실행 회원정보 주소 저장 을해 놨음 어플 실행 로그인 부안 물어보다 를

## 3) TFIDFVectorizer

In [17]:
vectorizer = TfidfVectorizer()
tfidf_vec = vectorizer.fit_transform(sum_text_list)
print(vectorizer.vocabulary_)
print(tfidf_vec.toarray())

{'배달': 1438, '리뷰': 1087, '자주': 2774, '참고': 3192, '편입': 3472, '한가지': 3571, '건의': 184, '사항': 1718, '최신': 3269, '주문': 3013, '자하': 2778, '메뉴': 1208, '확인': 3698, '하다': 3551, '기능': 444, '검색': 196, '특정': 3427, '문수': 1310, '않다': 2136, '보기': 1515, '위해': 2491, '래그': 1034, '시간': 1971, '소요': 1883, '되다': 924, '효율': 3727, '발생': 1415, '긍정': 439, '검토': 202, '주심': 3024, '과거': 330, '목록': 1266, '분명': 1579, '가게': 1, '시키다': 1998, '기억': 465, '먹다': 1199, '맞다': 1174, '싶다': 2054, '찾기': 3204, '곱창': 315, '치면': 3306, '뜨다': 1015, '화면': 3688, '전체': 2879, '포장': 3490, '탭을스': 3394, '크롤': 3361, '내리다': 591, '볼때': 1547, '아래': 2083, '로스': 1067, '자꾸만': 2758, '왼쪽': 2395, '오른쪽': 2341, '넘어가다': 609, '보다': 1519, '정말': 2918, '마트': 1121, '탭뭐': 3393, '하나': 3545, '선택': 1838, '좌우': 3002, '넘기다': 606, '가끔': 12, '도아': 866, '고스': 279, '한번': 3581, '쓰다': 2069, '정렬': 2915, '지역': 3102, '따르다': 976, '추가': 3276, '별도': 1503, '체크': 3245, '되어다': 927, '누르다': 643, '가격': 2, '대별': 775, '금액': 432, '도로': 846, '받다': 1409, '야합': 2178, '최근': 3262, '업데이트':

### 4) 학습 데이터셋

In [ ]:
X = tfidf_vec
y = reply_data_set["reaction"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 22)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(800, 3762) (200, 3762) (800,) (200,)


### 5) 데이터 학습

In [22]:
lr_model = LogisticRegression(random_state = 0)
lr_model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,0
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [23]:
y_pred = lr_model.predict(X_test)
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1]


### 6) 성능평가

In [26]:
print(f"Accuracy : {accuracy_score(y_test, y_pred)}")
print(f"Precision : {precision_score(y_test, y_pred)}")
print(f"Recall : {recall_score(y_test, y_pred)}")
print(f"F1-Score : {f1_score(y_test, y_pred)}")

Accuracy : 0.745
Precision : 0.7222222222222222
Recall : 0.22033898305084745
F1-Score : 0.33766233766233766


### 7) 예측

In [ ]:
new_review = "토종 순대국밥 집보다 맛있고 퀄리티가 높음, 안에 고기가 잡내 없이 고소함,  셀프바 음료수 무"
new_review = re.sub("[ㄱ-ㅎㅏ-ㅣ\s!@#$%^&*()]", "", new_review)

temp_word_list = okt.pos(new_review, stem = True)
new_words = []
for tex, pos in temp_word_list :
    if (pos in ['Noun', 'Verb']) & (tex not in stopwords) :
        word_list.append(tex)

new_words.append(" ".join(word_list))

X_new = vectorizer.transform(new_words)
print(X_new.toarray())

pred_new = lr_model.predict(X_new)
print(pred_new)

[[0. 0. 0. ... 0. 0. 0.]]
[0]


<>:2: DeprecationWarning: invalid escape sequence '\s'
<>:2: DeprecationWarning: invalid escape sequence '\s'
C:\Users\user\AppData\Local\Temp\ipykernel_10732\1155763619.py:2: DeprecationWarning: invalid escape sequence '\s'
  new_review = re.sub("[ㄱ-ㅎㅏ-ㅣ\s!@#$%^&*()]", "", new_review)
